# Defalut Credit Card 💳
---

In [ ]:
!pip install scikeras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

## [1] Load Data
---


In [3]:
colab_path = '/content/drive/MyDrive/Colab Notebooks'
file_path = colab_path + '/python-mldlnl/natural-language-processing/Data/default of credit card clients.csv'

data = pd.read_csv(file_path, header=1).drop(columns="ID")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   LIMIT_BAL                   30000 non-null  int64
 1   SEX                         30000 non-null  int64
 2   EDUCATION                   30000 non-null  int64
 3   MARRIAGE                    30000 non-null  int64
 4   AGE                         30000 non-null  int64
 5   PAY_0                       30000 non-null  int64
 6   PAY_2                       30000 non-null  int64
 7   PAY_3                       30000 non-null  int64
 8   PAY_4                       30000 non-null  int64
 9   PAY_5                       30000 non-null  int64
 10  PAY_6                       30000 non-null  int64
 11  BILL_AMT1                   30000 non-null  int64
 12  BILL_AMT2                   30000 non-null  int64
 13  BILL_AMT3                   30000 non-null  int64
 14  BILL_A

In [4]:
X = data.drop(columns="default payment next month")
y = data['default payment next month']

In [5]:
category = ["SEX", "EDUCATION", "MARRIAGE"]

for i in category:
    X[i] = X[i].astype("object")

## [2] Preprocessing
---

In [6]:
def preprocessing(df:pd.DataFrame):
    columns = df.columns

    for col in columns:
        if df[col].dtype == "object":
            ohe = OneHotEncoder(sparse=False)
            ohe_arr = ohe.fit_transform(df[col].values.reshape(-1, 1))
            ohe_df = pd.DataFrame(ohe_arr)

            df = pd.concat([df, ohe_df], axis=1)
            df = df.drop(columns=col)
        else:
            pass

    df = MinMaxScaler().fit_transform(df)

    return df

In [7]:
X_pre = preprocessing(X)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


## [3] PCA
---

In [8]:
pca = PCA(n_components=0.95).fit(X_pre)

X_pca = pca.transform(X_pre)

In [9]:
# 주성분 행렬
print("주성분 행렬 : ", pca.singular_values_)

# 주성분에 투영 후 분산 정도 설명
print("분산 정도 : ", pca.explained_variance_)

# 주성분에 투영 후 분산 정도 설명
print("분산 정도 비율 : ", pca.explained_variance_ratio_)

주성분 행렬 :  [125.9595317  119.21788538 107.88352473  77.37351817  41.88450817
  26.53589807  23.47956079]
분산 정도 :  [0.52887775 0.47377927 0.38797476 0.19956203 0.05847902 0.02347258
 0.01837694]
분산 정도 비율 :  [0.29966878 0.26844929 0.21983138 0.11307436 0.03313495 0.01329986
 0.0104126 ]


## [4] train test split
---

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, stratify=y, random_state=42
)

## [5] Grid Search CV
---

In [14]:
def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

In [ ]:
clf = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    hidden_layer_dim=100
)

clf.fit(X, y)

In [ ]:
params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1]
}

gs = GridSearchCV(
    clf, params, refit=False, cv=3, scoring="accuracy"
)

gs.fit(X, y)